# Stationarity and Ergodicity


## Overview

In this lecture we discuss stability and equilibrium behavior for continuous
time Markov chains.

To give one example of why this theory matters, consider queues, which are
often modeled as continuous time Markov chains.

Queueing theory is used in applications such as 

* treatment of patients arriving at a hospital
* optimal design of manufacturing processes 
* requests to a file server 
* air traffic
* customers waiting on a help line

A key topic in queueing theory is average behavior over the long run.

* Will the length of the queue grow without bounds?
* If not, is there some kind of long run equilibrium?
* If so, what is the average waiting time in this equilibrium?
* What is the average length of the queue over a week, or a month?



We will use the following imports

In [ ]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import quantecon as qe
from numba import njit

from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection


## Stationary Distributions

### Definition

Let $S$ be countable.

Recall that, for a discrete time Markov chain with Markov matrix $P$ on $S$, a
distribution $\psi$ is called stationary for $P$ if $\psi P = \psi$.

This means that if $X_t$ has distribution $\psi$, then so does $X_{t+1}$.

For continuous time Markov chains, the definition is analogous.

Given a Markov semigroup on $S$, a distribution $\psi \in \dD$ is called
**stationary** for $(P_t)$ if

$$
    \psi P_t = \psi 
    \text{ for all } t \geq 0
$$

As one example, we look {ref}`again <solvode>` at the chain on $S = \{0, 1,
2\}$ with intensity matrix

In [ ]:
Q = ((-3, 2, 1),
     (3, -5, 2),
     (4, 6, -10))

The following figure was shown before, except that now there is a black dot
that the three trajectories seem to be converging to.

(Recall that, in the color scheme, trajectories cool as time evolves.)

In [ ]:
def unit_simplex(angle):
    
    fig = plt.figure(figsize=(8, 6))
    ax = fig.add_subplot(111, projection='3d')

    vtx = [[0, 0, 1],
           [0, 1, 0], 
           [1, 0, 0]]
    
    tri = Poly3DCollection([vtx], color='darkblue', alpha=0.3)
    tri.set_facecolor([0.5, 0.5, 1])
    ax.add_collection3d(tri)

    ax.set(xlim=(0, 1), ylim=(0, 1), zlim=(0, 1), 
           xticks=(1,), yticks=(1,), zticks=(1,))

    ax.set_xticklabels(['$(1, 0, 0)$'], fontsize=12)
    ax.set_yticklabels(['$(0, 1, 0)$'], fontsize=12)
    ax.set_zticklabels(['$(0, 0, 1)$'], fontsize=12)

    ax.xaxis.majorTicks[0].set_pad(15)
    ax.yaxis.majorTicks[0].set_pad(15)
    ax.zaxis.majorTicks[0].set_pad(35)

    ax.view_init(30, angle)

    # Move axis to origin
    ax.xaxis._axinfo['juggled'] = (0, 0, 0)
    ax.yaxis._axinfo['juggled'] = (1, 1, 1)
    ax.zaxis._axinfo['juggled'] = (2, 2, 0)
    
    ax.grid(False)
    
    return ax

Q = np.array(Q)
ψ_00 = np.array((0.01, 0.01, 0.99))
ψ_01 = np.array((0.01, 0.99, 0.01))
ψ_02 = np.array((0.99, 0.01, 0.01))

ax = unit_simplex(angle=50)    

def flow_plot(ψ, h=0.001, n=400, angle=50):
    colors = cm.jet_r(np.linspace(0.0, 1, n))

    x_vals, y_vals, z_vals = [], [], []
    for t in range(n):
        x_vals.append(ψ[0])
        y_vals.append(ψ[1])
        z_vals.append(ψ[2])
        ψ = ψ @ expm(h * Q)

    ax.scatter(x_vals, y_vals, z_vals, c=colors, s=20, alpha=0.2, depthshade=False)

flow_plot(ψ_00)
flow_plot(ψ_01)
flow_plot(ψ_02)

# Add stationary distribution
#P_1 = expm(Q)

#mc = qe.MarkovChain(P_1)
#ψ_star = mc.stationary_distributions
#ax.scatter(0.2, 0.2, 0.6, c='k', s=20, alpha=0.2, depthshade=False)

plt.show()

(Suggests ergodicity.)


### Stationarity via the Generator

In many cases, it is easier to use the generator of the semigroup to identify
stationary distributions rather than the semigroup itself.

This is analogous to the idea that a point $\bar x$ in $\RR^d$ is stationary for
a vector ODE $x'_t = F(x_t)$ when $F(\bar x) = 0$.

(Here $F$ is the infinitesimal description, and hence analogous to the generator.)

The next result holds true under weaker conditions but the version stated here
is easy to prove and sufficient for applications we consider.


```{proof:theorem}
:label: statfromq

Let $(P_t)$ be a UC Markov semigroup with generator $Q$.  A distribution
$\psi$ on $S$ is stationary for $(P_t)$ if and only if $\psi Q = 0$.
```

```{proof:proof}
Fix $\psi \in \dD$ and suppose first that $\psi Q = 0$.

Since $(P_t)$ is a UC Markov semigroup, we have $P_t = e^{tQ}$ for all $t$,
and hence, for any $t \geq 0$,

$$
    \psi e^{tQ} = \psi + t \psi Q + t^2 \frac{\psi Q^2}{2!} 
    + \cdots
$$

From $\psi Q = 0$ we get $\psi Q^k = 0$ for all $k \in \NN$, so the last 
display yields $\psi P_t = \psi$.

Hence $\psi$ is stationary for $(P_t)$.

Now suppose that $\psi$ is stationary for $(P_t)$ and set $D_t := (1/t) (P_t -
I)$.

From the triangle inequality and the definition of the operator norm, for any given $t$,

$$
    \| \psi Q \| 
    \leq \| \psi (Q - D_t) \| + \| \psi D_t \|
    \leq \| Q - D_t \| + \| \psi D_t \|
$$

Since $(P_t)$ is UC and $Q$ is its generator, we have $\| D_t - Q \| \to 0$ in
$\lL(\ell_1)$ as $t \to 0$.

Hence $\| \psi Q \| \leq \liminf_{t \to 0} \| \psi D_t \|$.

As $\psi$ is stationary for $(P_t)$, we have $\psi D_t = 0$ for all $t$.

Hence $\psi Q = 0$, as was to be shown.
```



## Irreducibility and Uniqueness

Let $(P_t)$ be a Markov semigroup on $S$.

We say that state $y$ is **accessible** from state $x$ if
there exists a $t \geq 0$ such that $P_t(x, y) > 0$.

A Markov semigroup $(P_t)$ on $S$ is called **irreducible** if $y$ is
accessible from $x$ for every $(x,y) \in S \times S$.

We seek a characterization of irreducibility of $(P_t)$ in terms of its
generator.

As a first step, we will say there is a **$Q$-positive probability flow** from $x$
to $y$ if there exists a finite sequence $(z_i)_{i=0}^m$ in $S$ starting at
$x=z_0$ and ending at $y=z_m$ such that $Q(z_i, z_{i+1}) > 0$ for all $i$.


```{proof:theorem}
:label: equivirr

Let $(P_t)$ be a UC Markov semigroup with generator $Q$.
For distinct states $x$ and $y$, the following statements are equivalent:

1. The state $y$ is accessible from $x$ under $(P_t)$.
1. There is a $Q$-positive probability flow from $x$ to $y$.
1. $P_t(x, y) > 0$ for all $t > 0$.
```


```{proof:proof}
Pick any two distinct states $x$ and $y$.

It is obvious that statement 3 implies statement 1, so we need only prove that
1 implies 2 and 2 implies 3.

Starting with (1 $\implies$ 2), recall that

$$
    P_t(x, y) = t Q(x,y) + \frac{t^2}{2!} Q^2(x, y) + \cdots
$$ (ptexpan)

If $x$ is accessible from $y$, then $P_t(x, y) > 0$ for some $t > 0$, so
$Q^k(x,y) > 0$ for at least one $k \in \NN$.

Writing out the matrix product as a sum, we now have

$$
    Q^k(x,y) =
    \sum_{z_1}
    \sum_{z_2}
    \cdots
    \sum_{z_{k-1}}
        Q(x, z_1) Q(z_1, z_2) \cdots Q(z_{k-1}, y) 
        > 0
$$  (qkassum)

It follows that at least one element of the sum must be strictly positive.

Therefore, $Q$-positive probability flow from $x$ to $y$ exists.

Turning to (2 $\implies$ 3), suppose there is $Q$-positive probability flow
$(z_i)_{i=0}^m$ from $x$ to $y$.

We can and do assume that all elements of this sequence are distinct.

Let $(\lambda, K)$ be the jump chain pair constructed from $Q$ via
{eq}`lambdafromq`, {eq}`kfromqxx` and {eq}`kfromqxy`.

Note that, for each $i$, we have $Q(z_i, z_{i+1}) > 0$, so $\lambda(z_i) > 0$.

As a consequence, applying {eq}`kfromqxy`, we have 

$$
    K(z_i, z_{i+1}) = \frac{Q(z_i, z_{i+1})}{\lambda(z_i)} > 0
$$

It follows that $K^m(x, y) > 0$.

Let $(X_t)$ be the Markov chain constructed from $(\lambda, K)$ via
{proof:ref}`ejc_algo` and initial condition $x$.

Let $(Y_k)$ be the embedded discrete time jump chain and let $(J_k)$ be the jump
times

Let $M_t := \max \{k \in \NN \,:\, J_k \leq t\}$ be the number of jumps prior
to $t$.

Pick any $t > 0$ and observe that

$$

    P_t(x, y) 
    \geq \PP\{ X_t = y, \, M_t = m\}
    = \PP\{ Y_m = y, \, M_t = m\}
    = K^m(x, y) \PP\{ M_t = m\}
$$

Since $K^m(x, y) > 0$, we need only show that $\PP\{ N_t = m\} > 0$.  

But this is certainly true --- the details are a solved exercise.


```

{proof:ref}`equivirr` leads directly to the following strong result.

```{proof:corollary}
:label: perimposs
For a UC Markov semigroup $(P_t)$, the following statements are
equivalent:

1. $(P_t)$ is irreducible.
1. $P_t(x,y) > 0$ for all $t > 0$ and all $(x,y) \in S \times S$.
```


```{note}
To obtain stable long run behavior in discrete time Markov chains, it is
common to assume that the chain is aperiodic.

This needs to be assumed on top of irreducibility if one wishes to rule out
all dependence on initial conditions.

{proof:ref}`perimposs` shows that this is not a concern for irreducible
continuous time Markov chains.

They cannot be periodic, since positive probability flow from $x$ to $y$ at some 
$t > 0$ immediately implies positive flow for all $t > 0$.

```


Irreducible implies uniqueness: $P_t$ is a strict contraction for all $t > 0$.



## Asymptotic Stabiltiy

To be added.

Include Foguel alternative?

Thm 6.2 of RR and MTK.

## Exercises

### Exercise 1

Let $(P_t)$ be a Markov semigroup.  True or false:
for this semigroup, every state $x$ is accessible from itself. 



### Exercise 2

Let $(\lambda_k)$ be a sequence in $(0, \infty)$ and let $(W_k)$ be an independent
sequence such that $W_k \sim \Exp(\lambda_k)$.

A **pure birth process** starting at zero is a continuous time Markov process
$(N_t)$ defined by 

* $J_0 = 0$
* $J_{k+1} = J_k + W_{k+1}$ and
* $N_t := \max \{k \in \NN \,:\, J_k \leq t\}$.

Find the intensity matrix.

$$
Q=\begin{pmatrix}
    -\lambda_0 & \lambda_0 & 0 & 0 & \cdots \\
    0 & -\lambda_1 & \lambda_1 & 0 & \cdots \\
    0 & 0 & -\lambda_2 & \lambda_2 & \cdots\\
    \vdots & \vdots & \vdots & & \vdots \ddots
\end{pmatrix}
$$

Show that $N_t$ supported on all of $\ZZ_+$ for any $t > 0$.

Show that $(P_t)$, the corresponding Markov semigroup, has no stationary
distribution.



## Solutions

### Solution to Exercise 1

The statement is true.  With $t=0$ we have $P_t(x,x) = I(x,x) = 1 > 0$.